# INTELIVIX CHALLENGE (MID LEVEL)

No NLTK (Natural Language Toolkit), uma plataforma para desenvolvimento de aplicações voltada para Processamento de Linguagem Natural para Python, estão disponíveis alguns corpora com textos já classificados, "taggeados", etc. Um deles, o corpus “Brown”, reune 500 textos de diferentes categorias.

### O desafio consiste em:
* Identificar as 2 categorias mais frequentes, extrair os textos pertencentes a cada uma dessas categorias;
* Criar uma base de treino (66% do total dos textos) e uma base de teste (os 34% restantes); Os textos devem ser distribuídos aleatoriamente em ambas as bases;
* Criar um classificador capaz de categorizar o conjuntos de teste.
* O treinamento deve possuir duas etapas:

### Pré-processamento:
* Extrair tokens, eliminando pontuações, stopwords e realizando stemming (ou stemização) nos termos restantes;
* Os textos de treinamento, representados por listas dos tokens restantes, devem ser convertidos em uma matriz TF-IDF (Text Frequency – Inverse Document Frequency). (Implementar a parte de IDF do algoritmo caso não encontre similar em outra biblioteca.)
* Classificação:
* Cada linha da matriz, que representa um documento da base de treinamento, deve ser apresentada a um classificador juntamente com sua categoria, de forma que ocorra o aprendizado.	O teste do classificador deve seguir o mesmo raciocínio.

### Sobre a entrega:
* Deve-se escolher 3 diferentes classificadores, treiná-los, testá-los e reportar os resultados, comparando-os e escolhendo o melhor, justificando a escolha.
* Para as comparações, deve-se calcular uma matriz de confusão para cada classificador.
* Evidentemente, outras métricas adicionais que, por ventura, sejam consideradas necessárias, podem ser utilizadas.
* Os códigos e o relatório devem ser entregues em um ipython notebook, o qual deve ser auto-suficiente para ser executado (assumindo que o computador a executar possua todas as ferramentas necessárias instaladas).

### Ferramentas sugeridas:
* NLTK (pré-processamento – alguns dos passos estão implementados, outros não);
* scikit-learn (classificação).

In [1]:
import string
import re
import random

from operator import itemgetter

from nltk.corpus import brown, stopwords
from nltk.stem.porter import *

from scipy.stats import *

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier

In [2]:
def most_used_categories(corpus):
    most_used = []    
    for genre in corpus.categories():
        most_used.append((genre, len(corpus.fileids(categories=genre))))
    most_used.sort(key=itemgetter(1), reverse=True)
    return [most_used[0][0], most_used[1][0]]


def tokenize_and_stem_text(text):
    text = [(re.sub(r'\d+', '', word.translate(str.maketrans('', '', string.punctuation)))).lower() for word in text]
    text = [stemmer.stem(word) for word in text if word not in stopwords.words('english') and word != '']
    return set(text)


def train_and_test_classifier(classifier):
    classifier.fit(X_train_transformed, y_train)
    print ("Test accuracy:", classifier.score(X_test_transformed, y_test))

In [3]:
corpus = brown
categories = []

stemmer = PorterStemmer()
tfidf = TfidfVectorizer(max_df=0.5)

dataset = []
inputset = []
outputset = []

categories = most_used_categories(corpus)
print(categories)

for category in categories:
    for fileid in corpus.fileids(categories = category):
        dataset.append([corpus.words(fileids = fileid), category])

for i in range(len(dataset)):
    dataset[i][0] = tokenize_and_stem_text(dataset[i][0])
    inputset.append(' '.join(map(str, dataset[i][0])))
    outputset.append(dataset[i][1])

['learned', 'belles_lettres']


In [4]:
X_train, X_test, y_train, y_test = train_test_split(inputset, 
                                                    outputset, 
                                                    test_size=0.34, 
                                                    random_state=42, 
                                                    shuffle=True)

In [5]:
X_train_transformed = tfidf.fit_transform(X_train)
X_test_transformed = tfidf.transform(X_test)


tree_model = DecisionTreeClassifier(random_state=42)

svm_model = svm.SVC(C=0.8, 
                    random_state=1234,
                    kernel='linear')

mlp_model = MLPClassifier(hidden_layer_sizes=(35, ), 
                          activation="relu", 
                          solver="adam", 
                          alpha=0.65, 
                          max_iter=1000, 
                          random_state=1234)

In [6]:
# Decision Tree
train_and_test_classifier(tree_model)

Test accuracy: 0.603773584906


In [11]:
# Support Vector Machine
train_and_test_classifier(svm_model)

Test accuracy: 0.830188679245


In [12]:
# Multi-Layer Perceptron
train_and_test_classifier(mlp_model)

Test accuracy: 0.867924528302


In [13]:
tree_predicted = tree_model.predict(X_test_transformed)
svm_predicted = svm_model.predict(X_test_transformed)
mlp_predicted = mlp_model.predict(X_test_transformed)

In [14]:
print("DECISION TREE")
#print(metrics.classification_report(y_test, tree_predicted))
print(metrics.confusion_matrix(y_test, tree_predicted))

print("\nSUPPORT VECTOR MACHINE")
#print(metrics.classification_report(y_test, svm_predicted))
print(metrics.confusion_matrix(y_test, svm_predicted))

print("\nMULTI-LAYER PERCEPTRON")
#print(metrics.classification_report(y_test, mlp_predicted))
print(metrics.confusion_matrix(y_test, mlp_predicted))

DECISION TREE
[[16  9]
 [12 16]]

SUPPORT VECTOR MACHINE
[[24  1]
 [ 8 20]]

MULTI-LAYER PERCEPTRON
[[23  2]
 [ 5 23]]


# Comentários

* Palavras com frequência acima de 50% não foram selecionados para a matriz TF-IDF.
* Considerando a matriz de confusão e a taxa de classificação, a MLP seria o modelo escolhido apesar do maior tempo de treinamento. Contudo, é necessário conduzir testes estatísticos nos resultados dos dois modelos (além de realizar outros ajustes nos parâmetros) para verificar se o ganho da MLP em relação a SVM é relevante para justificar a escolha de um modelo com maior tempo de processamento.
* Os parâmetros da SVM e da MLP foram escolhidos de forma empírica.